## 1. Import the necessary libraries and data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Activation, Dropout
from tensorflow.keras.layers import Dense, Reshape, Conv1D

In [ ]:
inp0 = pd.read_csv('/kaggle/input/gamestop-historical-stock-prices/GME_stock.csv')

inp0 = inp0.sort_values(by='date',ascending=True)
print(inp0.shape)
inp0.head()

## 2. Plot the time series

In [ ]:
inp0.plot("date", "close_price")
plt.show()

In [ ]:
# Extract the values for Close from the DataFrame
ts_data = inp0.close_price.values.reshape(-1,1)

plt.figure(figsize=[14,5])
plt.plot(ts_data)
plt.show()

## 3. Assign the final 25% data as test data and the first 75% as train data

In [ ]:
train_recs = int(len(ts_data)*0.75)
train_data = ts_data[:train_recs]
test_data = ts_data[train_recs:]

len(train_data), len(test_data)

## 4. Scale the train and test data

In [ ]:
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data)
test_scaled = scaler.transform(test_data)

## 5. Get the lookback data for the train and test

In [ ]:
def get_lookback(inp, look_back):
  y = pd.DataFrame(inp)
  dataX = [y.shift(i) for i in range(1, look_back+1)]
  dataX = pd.concat(dataX, axis=1)
  dataX.fillna(0, inplace = True)
  return dataX.values, y.values

In [ ]:
look_back = 10
trainX, trainY = get_lookback(train_scaled, look_back=look_back)
testX, testY = get_lookback(test_scaled, look_back= look_back)
trainX.shape, testX.shape

## 6. Build a model

In [ ]:
model = Sequential()

model.add(Reshape((look_back,1),input_shape=(look_back,)))
model.add(Conv1D(5, 3, activation='relu'))
model.add(SimpleRNN(32))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('linear'))

model.summary()

## 7. Compile and Fit the model

In [ ]:
model.compile(loss='mean_squared_error',optimizer='adam')

model.fit(trainX,trainY, epochs=10,batch_size=18,verbose=2,validation_split=0.1)

## 8. RMSE of the model

In [ ]:
def get_model_perf(model_obj):
  score_train = model_obj.evaluate(trainX, trainY, verbose=0)
  print('Train RMSE: %.2f RMSE' % (math.sqrt(score_train)))
  score_test = model_obj.evaluate(testX, testY, verbose=0)
  print('Test RMSE: %.2f RMSE' % (math.sqrt(score_test)))

In [ ]:
get_model_perf(model)

## 9. Plot the predictions

In [ ]:
def plot_pred(model_obj):
  testPredict = scaler.inverse_transform(model_obj.predict(testX))
  pred_test_plot = ts_data.copy()
  pred_test_plot[:train_recs+look_back,:] = np.nan
  pred_test_plot[train_recs+look_back:,:] = testPredict[look_back:]
  plt.plot(ts_data, label='Actual')
  plt.plot(pred_test_plot, "--", label='Predicted')
  plt.legend()

In [ ]:
def plot_pred_zoom(model_obj):
  testPredict = scaler.inverse_transform(model_obj.predict(testX))
  pred_test_plot = testPredict
  plt.plot(ts_data[train_recs:], label='Actual')
  plt.plot(pred_test_plot, "--", label='Predicted')
  plt.legend()

In [ ]:
#the entire view
plt.figure(figsize=[12,5])
plot_pred(model)

In [ ]:
#the zoomed-in view
plt.figure(figsize=[12,6])
plot_pred_zoom(model)